# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacationPy_db = pd.read_csv('output_data/city_data.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [13]:
locations = vacationPy_db[['Lat', 'Lng']]
humidity = vacationPy_db["Humidity"]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                              dissipating=False, max_intensity=350,
                              point_radius = 6)
    
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
hotel_df = vacationPy_db[(vacationPy_db["Max Temp"] <= 85) & (vacationPy_db["Max Temp"] > 65)&(vacationPy_db["Humidity"] <= 30) & (vacationPy_db["Cloudiness"] <= 20)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,kingman,35.19,-114.05,73.40,11,1,4.70,US,1601396723
37,saint george,37.10,-113.58,69.01,15,1,3.40,US,1601396629
59,abha,18.22,42.51,77.00,13,7,9.17,SA,1601397022
182,windhoek,-22.56,17.08,81.00,6,0,4.70,NaN,1601396780
197,magalia,39.81,-121.58,78.80,18,5,4.09,US,1601396786
234,mount isa,-20.73,139.50,69.80,26,0,4.70,AU,1601396799
290,dutlwe,-23.98,23.90,75.81,12,0,9.06,BW,1601396819
327,dodge city,37.75,-100.02,68.00,22,1,9.17,US,1601396835
404,calama,-22.47,-68.93,71.60,10,0,14.99,CL,1601396863
439,snyder,32.72,-100.92,68.00,20,1,11.41,US,1601396875


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df["hotel_name"] = "" 
hotel_df

C:\Users\jcarrillo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel_name
26,kingman,35.19,-114.05,73.40,11,1,4.70,US,1601396723,
37,saint george,37.10,-113.58,69.01,15,1,3.40,US,1601396629,
59,abha,18.22,42.51,77.00,13,7,9.17,SA,1601397022,
182,windhoek,-22.56,17.08,81.00,6,0,4.70,NaN,1601396780,
197,magalia,39.81,-121.58,78.80,18,5,4.09,US,1601396786,
234,mount isa,-20.73,139.50,69.80,26,0,4.70,AU,1601396799,
290,dutlwe,-23.98,23.90,75.81,12,0,9.06,BW,1601396819,
327,dodge city,37.75,-100.02,68.00,22,1,9.17,US,1601396835,
404,calama,-22.47,-68.93,71.60,10,0,14.99,CL,1601396863,
439,snyder,32.72,-100.92,68.00,20,1,11.41,US,1601396875,


In [17]:
params = {
    "radius": 5000, 
    "types": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    params["location"] = f'{row["Lat"]},{row["Lng"]}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        hotel_df.loc[index,'hotel_name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.") 
hotel_df

C:\Users\jcarrillo\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel_name
26,kingman,35.19,-114.05,73.40,11,1,4.70,US,1601396723,Rodeway Inn Kingman Route 66
37,saint george,37.10,-113.58,69.01,15,1,3.40,US,1601396629,Best Western Plus Abbey Inn
59,abha,18.22,42.51,77.00,13,7,9.17,SA,1601397022,Abha Hotel
182,windhoek,-22.56,17.08,81.00,6,0,4.70,NaN,1601396780,Hotel Uhland
197,magalia,39.81,-121.58,78.80,18,5,4.09,US,1601396786,Kris West Park
234,mount isa,-20.73,139.50,69.80,26,0,4.70,AU,1601396799,ibis Styles Mt Isa Verona
290,dutlwe,-23.98,23.90,75.81,12,0,9.06,BW,1601396819,
327,dodge city,37.75,-100.02,68.00,22,1,9.17,US,1601396835,Windsor Inn & Suites
404,calama,-22.47,-68.93,71.60,10,0,14.99,CL,1601396863,Park Hotel Calama
439,snyder,32.72,-100.92,68.00,20,1,11.41,US,1601396875,Holiday Inn Express & Suites Snyder


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))